In [261]:
import pandas as pd


In [262]:
# ------------------ #
# Importing Packages #
# ------------------ #
import openmeteo_requests
import pandas as pd
import plotly.express as px
import seaborn as sns
import requests_cache
from retry_requests import retry
import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, date, timedelta



# --------------- #
# Open-meteo API  #
# --------------- #


# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://api.open-meteo.com/v1/forecast"
params = {
	"latitude": 52.37,
		"longitude": 4.89,
	"hourly": ["temperature_2m", "relative_humidity_2m", "dew_point_2m", "precipitation"],
	"models": "knmi_seamless",
	"timezone": "Europe/Berlin",
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates: {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation: {response.Elevation()} m asl")
print(f"Timezone: {response.Timezone()}{response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0: {response.UtcOffsetSeconds()}s")

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
hourly_relative_humidity_2m = hourly.Variables(1).ValuesAsNumpy()
hourly_dew_point_2m = hourly.Variables(2).ValuesAsNumpy()
hourly_precipitation = hourly.Variables(3).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}

hourly_data["temperature_2m"] = hourly_temperature_2m
hourly_data["relative_humidity_2m"] = hourly_relative_humidity_2m
hourly_data["dew_point_2m"] = hourly_dew_point_2m
hourly_data["precipitation"] = hourly_precipitation

hourly_dataframe = pd.DataFrame(data = hourly_data)
print("\nHourly data\n", hourly_dataframe)

# ---------------------------------------- End

# --------------- #
#     Figures     #
# --------------- #

#Figure code here





# ---------------------------------------- End

# --------------- #
#    Streamlit    #
# --------------- #

st.title("KNMI Weerdata")
# Sidebar inputs
st.sidebar.header("Location & Settings")
latitude = st.sidebar.number_input("Latitude", value=52.52, format="%.4f")
longitude = st.sidebar.number_input("Longitude", value=13.41, format="%.4f")
timezone = st.sidebar.text_input("Timezone", "Europe/Berlin") 

st.dataframe(hourly_dataframe)

st.write("This is a test")
st.write("GODOOODOAOGAGOA")

# ---------------------------------------- End

2025-09-25 10:16:19.639 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-25 10:16:19.641 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-25 10:16:19.643 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-25 10:16:19.644 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-25 10:16:19.646 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-25 10:16:19.646 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-25 10:16:19.647 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-25 10:16:19.649 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

Coordinates: 52.36600112915039°N 4.901000022888184°E
Elevation: 11.0 m asl
Timezone: b'Europe/Berlin'b'GMT+2'
Timezone difference to GMT+0: 7200s

Hourly data
                          date  temperature_2m  relative_humidity_2m  \
0   2025-09-24 22:00:00+00:00       13.767500                  61.0   
1   2025-09-24 23:00:00+00:00       13.367499                  61.0   
2   2025-09-25 00:00:00+00:00       12.867499                  59.0   
3   2025-09-25 01:00:00+00:00       12.817500                  60.0   
4   2025-09-25 02:00:00+00:00       12.167500                  63.0   
..                        ...             ...                   ...   
163 2025-10-01 17:00:00+00:00       12.400000                  61.0   
164 2025-10-01 18:00:00+00:00       11.550000                  64.0   
165 2025-10-01 19:00:00+00:00       10.800000                  66.0   
166 2025-10-01 20:00:00+00:00       10.150000                  67.0   
167 2025-10-01 21:00:00+00:00        9.500000              

In [263]:

# Convert 'vandaag' from yyyy-mm-dd to mm-dd format
vandaag_mmdd = datetime.now().strftime("%m-%d")
jaar = vandaag.year
print("vandaag: "+vandaag_mmdd)


#local rainfall data "schellingwoude"
neerslag_df = pd.read_csv("CSV_SCHELLINGWOUDE.csv", sep=";")
neerslag_df["Datum"] = pd.to_datetime(neerslag_df["Datum"], format = "%Y%m%d")
neerslag_df = neerslag_df.rename(columns={"Column2":"Neerslag"})
neerslag_df.set_index("Datum", inplace=True)
neerslag_df = neerslag_df[neerslag_df.index >= "1950-01-01"]
neerslag_df["Year"] = neerslag_df.index.year
neerslag_df["Day_Month"] = neerslag_df.index.strftime("%m-%d")
print(neerslag_df.head(6))


hourly_dataframe["Day_Month"] = hourly_dataframe["date"].dt.strftime("%m-%d")
#calculating expected rain from hourly data (API)
neerslag_huidig = pd.DataFrame()
neerslag_huidig["Day_Month"] = hourly_dataframe["date"].dt.strftime("%m-%d")
neerslag_huidig["Expected_Rain"] = hourly_dataframe.groupby("Day_Month")["precipitation"].sum()
neerslag_huidig["Expected_Rain"] = neerslag_huidig["Expected_Rain"].fillna(0)

if vandaag_mmdd in neerslag_huidig["Day_Month"].values:
    expected_rain_today = neerslag_huidig.loc[neerslag_huidig["Day_Month"] == vandaag_mmdd, "Expected_Rain"].values[0]


if vandaag_mmdd in neerslag_df["Day_Month"].values:
    historical_rain_today = neerslag_df.loc[neerslag_df["Day_Month"] == vandaag_mmdd].copy()
    historical_rain_today["Year"] = historical_rain_today.index.year

    
df_expected_rain = pd.DataFrame({
    "Day_Month": [vandaag_mmdd],
    "Expected_Rain": [expected_rain_today],
    "Jaar": [jaar]
})


print("HIER", historical_rain_today.head())


# Display the processed hourly data
print("\nNeerslag\n", neerslag_huidig) 

vandaag: 09-25
            Neerslag  Year Day_Month
Datum                               
1950-01-01       0.0  1950     01-01
1950-01-02       0.0  1950     01-02
1950-01-03      16.0  1950     01-03
1950-01-04      71.0  1950     01-04
1950-01-05       0.0  1950     01-05
1950-01-06      41.0  1950     01-06
HIER             Neerslag  Year Day_Month
Datum                               
1950-09-25     120.0  1950     09-25
1951-09-25       0.0  1951     09-25
1952-09-25     137.0  1952     09-25
1953-09-25      47.0  1953     09-25
1954-09-25      75.0  1954     09-25

Neerslag
     Day_Month  Expected_Rain
0       09-24            0.0
1       09-24            0.0
2       09-25            0.0
3       09-25            0.0
4       09-25            0.0
..        ...            ...
163     10-01            0.0
164     10-01            0.0
165     10-01            0.0
166     10-01            0.0
167     10-01            0.0

[168 rows x 2 columns]


In [393]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots


# Historical bars
fig = make_subplots(rows=1, cols=2,
shared_xaxes=True, 
shared_yaxes=True, 
column_widths=[1, 0.2],
subplot_titles=("Historische Neerslag", "Voorspelde Neerslag")
)

fig.add_trace(
    go.Bar(
        x=historical_rain_today["Year"],
        y=historical_rain_today["Neerslag"],
        name="Historisch",
        marker=dict(
            color=historical_rain_today["Neerslag"],  
            colorscale=px.colors.sequential.Blues,
            colorbar=dict(title="Neerslag"),
            showscale=True
        )
    ), row=1, col=1
)

# Predicted rainfall (salmon bar)
fig.add_trace(
    go.Bar(
        x=df_expected_rain["Jaar"],
        y=df_expected_rain["Expected_Rain"],
        name="Voorspeld",
        marker=dict(
            color="salmon"
        ),
        showlegend=True
    ), row=1, col=2
)

#trend line rainfall
fig.add_trace(    
    go.Scatter(
        x=historical_rain_today["Year"],
        y=historical_rain_today["Neerslag"].expanding().mean(),
        mode="lines",
        name="Gemiddelde",
        line=dict(color="black", width=1, dash = "dot", shape="spline")
    )
)

fig.add_hline(
    y = historical_rain_today["Neerslag"].mean(),
    line=dict(color="black", width=1, dash = "dot"),
    annotation_text = "Gemiddelde",
    annotation_position = "top right",
    row=1, col=2
)
# Highlight tick label for the current year
fig.update_layout(
    xaxis=dict(
        tickangle=90,   
        tickfont=dict(color="black"),  # default color
    ),
    yaxis_title="Neerslag (mm)",
    legend_title="Legenda",
   
)

fig.add_annotation(
    x=0.00,                   # x-position (can be axis value or fraction with xref)
    y=1.3,    
    xref = "paper",
    yref = "paper",               # y-position (can be axis value or fraction with yref)
    text="Vandaag: " + vandaag_mmdd, # Text to display
    showarrow=False,         # Show arrow pointing to the point
    font=dict(size=25)       # Font size
)
# Make the tick label for the prediction year red
fig.update_xaxes(
    tickvals=historical_rain_today.loc[
        historical_rain_today["Neerslag"] > 10, "Year"
    ].unique().tolist() + [df_expected_rain["Jaar"].iloc[0]],
    ticktext=historical_rain_today.loc[
        historical_rain_today["Neerslag"] > 10, "Year"
    ].astype(str).tolist(),
    tickangle=90  # optional: rotate for readability
)


#moving legend outside plot area
fig.update_traces(
    selector=dict(name="Historisch"),
    marker_color=historical_rain_today["Neerslag"],  # numeric
    marker_colorscale=px.colors.sequential.Blues,
    marker_colorbar=dict(
        title="Neerslag (mm)",
        x=1.02,          # horizontal position (right of plot)
        y=0.2,          # vertical position (center)
        len=0.8,        # length of the colorbar
    )
)

fig.show()
